In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve import internal
import netgen.gui

internal.visoptions.autoscale = False

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
mesh.GetBoundaries()

In [ ]:
def joinBounds(b1, b2):
    return b1+"|"+b2

In [ ]:
for bc in mesh.GetBoundaries():
    print(bc, "is top or right?:", bc=="right" or bc=="top")

In [ ]:
Dir = "bottom"
Neu = "left"
Rob = "right|top"
display([bc for bc in mesh.GetBoundaries()])
display([y if (bc=="top" or bc=="right") else 0 for bc in mesh.GetBoundaries()])

In [ ]:
f = 1
# (inhom) Dirichlet BC
ud = CoefficientFunction([x if bc=="bottom" else 0 for bc in mesh.GetBoundaries()])

# Not we set the coefficient functions for the Neumann and Robin BC..note the y --> CoefficientFunction defined by NGSolve
# Both Neumann and Robin depend on g
gRN = CoefficientFunction([y if bc!="bottom" else 0 for bc in mesh.GetBoundaries()])
gN = CoefficientFunction([y if bc=="left" else 0 for bc in mesh.GetBoundaries()])
gR = CoefficientFunction([y if (bc=="top" or bc=="right") else 0 for bc in mesh.GetBoundaries()])
# Robin boundary at top and right
alpha = 5
R_alpha = CoefficientFunction([alpha if (bc=="right" or  bc=="top") else 0 for bc in mesh.GetBoundaries()])
R_dalpha = CoefficientFunction([1/alpha if (bc=="right" or  bc=="top") else 0 for bc in mesh.GetBoundaries()])

In [ ]:
runtimes = {"primal": -1, "mixed": -1}

Mathematical framework formulation:

$\int_{\Omega}\nabla u \nabla v -5\int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N, \Gamma_R}yv$

For NGSolve:

*$\int_{\Omega}\nabla u \nabla v - R_{alpha} \int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N, \Gamma_R} g\ v$*

In [ ]:
from time import time_ns

order = 2
FESp = H1(mesh, order=order, dirichlet="bottom")
up, vp = FESp.TnT()

# Forms
ap = BilinearForm(FESp)
ap += grad(up)*grad(vp)*dx  - R_alpha*up*vp*ds #(definedon=mesh.Boundaries("right|top"))

fp = LinearForm(FESp)
fp += f*vp*dx + gN*vp*ds #(definedon=mesh.Boundaries("left"))

with TaskManager():
    ap.Assemble()
    fp.Assemble()

# Gridfunction
gfup = GridFunction(FESp, "u_primal")
gfup.Set(ud, BND) # BND =^ definedon=mesh.Boundaries("bottom|right|top|left")

# Need to take of the Dirichlet BC
with TaskManager():
    start_p = time_ns()
    # Homogenyze the system... "do-it-yoursolve" version
    # rhs = fp.vec.CreateVector()
    # rhs.data = fp.vec - ap.mat * gfup.vec
    # gfup.vec.data += ap.mat.Inverse(freedofs=FESp.FreeDofs()) * rhs
    # Should be same as
    solvers.BVP(bf=ap, lf=fp, gf=gfup)
    delta_p = (time_ns() - start_p) * 1e-9
    print(f"Runtime primal: {delta_p}s")
    runtimes["primal"] = round(delta_p, 3)

flux = CoefficientFunction(grad(gfup)) #R_alpha *

internal.visoptions.autoscale = False
Draw (gfup)
Draw (flux, mesh, "flux_primal")

# Mixed formulation: similar to Example 4.1


In [ ]:
# Setup the mixed method
order_flux = 2#order +1
V = HDiv(mesh, order=order_flux, dirichlet="left") # "Neumann is the new Dirichlet" 
Q = L2(mesh, order=order_flux-1) #, dirichlet="bottom"

FESm = FESpace([V,Q]) # Compound space
TnTm = FESm.TnT()
#print(TnT)
(sigma, um), (tau, vm) = TnTm # Unpacking in python is DOPE
#print((sigma, um), (tau, vm))
# Above is equivalent to ...
# sigma, um = FESm.TrialFunction()
# tau, vm = FESm.TestFunction()

# Setup the special function "normal"
# n = specialcf.normal(2) # from Philip
print("mesh.dim:", mesh.dim)
normal = specialcf.normal(mesh.dim)
print(normal)

# Set up forms again
am = BilinearForm(FESm)
am += (sigma*tau + div(sigma)*vm + div(tau)*um)*dx + (R_dalpha*(tau.Trace()*normal)*(sigma.Trace()*normal))*ds #(definedon=mesh.Boundaries("top|right"))
am.Assemble()

fm = LinearForm(FESm)
fm += -f*vm*dx + ud*(tau.Trace()*normal)*ds - gR*R_dalpha*tau.Trace()*normal*ds #(definedon=mesh.Boundaries("top|right"))
# ud*(tau.Trace()*normal)*ds(definedon=mesh.Boundaries("bottom"))
# + gR*(tau.Trace()*normal)*ds
fm.Assemble()

# Now on to the actual problem
gfm = GridFunction(FESm, name="gf_mixed")
gfsigma, gfum = gfm.components
# Remember g from before? :D
# It was defined as scalar valued --> "need to bring it into the vector world"
gfsigma.Set(normal*gN, BND) #definedon=mesh.Boundaries("left")) # definedon=mesh.Boundaries("left") / "bottom|right|top|left"
#gfum.Set(ud)

with TaskManager():
    start_m = time_ns()
    fm.vec.data -= am.mat * gfm.vec
    gfm.vec.data += am.mat.Inverse(freedofs=FESm.FreeDofs(), inverse="umfpack") * fm.vec
    #solvers.BVP(bf=am, lf=fm, gf=gfm)
    delta_m = (time_ns() - start_m) * 1e-9
    print(f"Runtime mixed: {delta_m}s")
    runtimes["mixed"] = round(delta_m, 3)

Draw (gfsigma, mesh, "flux_mixed")
Draw (gfum, mesh, "u_mixed")
internal.visoptions.autoscale = False
#Redraw()

In [ ]:
# "The" gridfunction...compound space of 2 spaces --> 2 components
gfmTest = GridFunction(FESm, name="gf_mixedTest")
gfsigmaTest, gfumTest = gfmTest.components

# Let's draw something just to see how it behaves
# gfsigmaTest.Set(CoefficientFunction( (sin(10*x), sin(10*y)) ))

gfsigma.Set(g*normal, definedon=BND)
gfumTest.Set (x)

# Draw (gfsigmaTest, mesh, "sinx_siny")
Draw (gfsigmaTest, mesh, "g*normal")
Draw (gfumTest, mesh, "x")